 # requiriments

In [1]:
import torch, torchvision
print(torch.__version__)        # Should show 2.4.0
print(torchvision.__version__)  # Should show 0.19.0

2.4.0+cu121
0.19.0+cu121


In [2]:
# base
import sys
import os
#cleaning
import re
#manupulate numbers
import numpy as np
import pandas as pd
import random
#Graph
import seaborn  as sns
import matplotlib.pyplot as plt
from datetime import date,datetime,timedelta
#store
import pickle
#method
#sys.path.append('..')
#from src.settings import Settings
#settings = Settings()
#embeds
import torch
import torch.nn.functional as F
import open_clip
#modeling
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
#ensamble
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [3]:
MODEL_ID = "hf-hub:timm/ViT-B-16-SigLIP-i18n-256"
model, preprocess = open_clip.create_model_from_pretrained(
    MODEL_ID,
    device='cuda',
    precision='fp16'
)
model.to('cuda')
model.eval()
tokenizer = open_clip.get_tokenizer(MODEL_ID)

open_clip_pytorch_model.bin:   0%|          | 0.00/1.48G [00:00<?, ?B/s]

open_clip_config.json:   0%|          | 0.00/891 [00:00<?, ?B/s]

/var/cache/pypoetry/virtualenvs/app-K4bO3-is-py3.10/lib/python3.10/site-packages/open_clip/factory.py:129: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.l

tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [5]:
def get_prompt_embeddings(prompt: str):
    text_input = tokenizer(prompt).to('cuda')  # Tokenize the labels
    with torch.no_grad():
        text_features = model.encode_text(text_input)

    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

    return text_features

 # Data

In [19]:
df = pd.read_csv('../data/train.csv')
df

,uid,city,description,homeType,latitude,longitude,garageSpaces,hasSpa,yearBuilt,numOfPatioAndPorchFeatures,lotSizeSqFt,avgSchoolRating,MedianStudentsPerTeacher,numOfBathrooms,numOfBedrooms,priceRange
0,1748,austin,MULTIPLE OFFERS submit best & final to Agent b...,Single Family,30.380089,-97.800621,0,False,1988,0,102366.0,7.000000,17,4.0,4,650000+
1,13380,austin,"4644 Hoffman Dr, Austin, TX 78749 is a single ...",Single Family,30.199486,-97.859947,0,False,1997,0,6534.0,6.666667,16,3.0,4,350000-450000
2,4115,austin,"6804 Canal St, Austin, TX 78741 is a single fa...",Single Family,30.227398,-97.696083,0,False,1952,0,5619.0,3.333333,11,1.0,2,0-250000
3,6926,austin,Beautiful large lot with established trees. Lo...,Single Family,30.205469,-97.792351,4,False,1976,0,6416.0,4.000000,14,2.0,4,0-250000
4,14480,austin,Stunning NW Hills designer remodel by Cedar an...,Single Family,30.345106,-97.767426,2,False,1984,0,10759.0,7.000000,16,3.0,5,650000+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,8614,austin,Updated Great Hills Opportunity! Large corner ...,Single Family,30.409927,-97.763390,2,False,1982,3,15246.0,7.000000,17,3.0,3,650000+
9996,10505,austin,Adorable 3/2 in the heart of South Austin! Sit...,Single Family,30.206074,-97.779472,0,False,1970,0,6577.0,4.000000,14,2.0,3,250000-350000
9997,6942,austin,"7322 Gaines Mill Ln, Austin, TX 78745 is a sin...",Single Family,30.198936,-97.810913,0,False,1981,1,7623.0,4.000000,13,2.0,3,350000-450000
9998,1845,austin,"5213 Doe Valley Ln, Austin, TX 78759 is a sing...",Single Family,30.391880,-97.761292,0,False,1979,2,13939.2,7.000000,17,3.0,3,250000-350000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   uid                         10000 non-null  int64  
 1   city                        10000 non-null  object 
 2   description                 9999 non-null   object 
 3   homeType                    10000 non-null  object 
 4   latitude                    10000 non-null  float64
 5   longitude                   10000 non-null  float64
 6   garageSpaces                10000 non-null  int64  
 7   hasSpa                      10000 non-null  bool   
 8   yearBuilt                   10000 non-null  int64  
 9   numOfPatioAndPorchFeatures  10000 non-null  int64  
 10  lotSizeSqFt                 10000 non-null  float64
 11  avgSchoolRating             10000 non-null  float64
 12  MedianStudentsPerTeacher    10000 non-null  int64  
 13  numOfBathrooms              1000

 ## feature selection

### objetive variable

In [20]:
priceRange_map = {i: priceRange for priceRange,i in df['priceRange'].value_counts().sort_values().reset_index().to_dict()['priceRange'].items()}
priceRange_map
maps_dict = {'priceRange': priceRange_map}
df['priceRange'] = df['priceRange'].map(maps_dict['priceRange'])

df[['priceRange']].describe().T

,count,mean,std,min,25%,50%,75%,max
priceRange,10000.0,2.2696,1.334726,0.0,1.0,2.0,3.0,4.0


### predictive features

In [22]:
ind = df[df['description'].isnull()==True]['description'].index
df[df['description'].isnull()==True]['description']

7016    NaN
Name: description, dtype: object

In [23]:
df.loc[ind,'description'] = ''

In [24]:
df[df['description'].isnull()==True]['description']

Series([], Name: description, dtype: object)

In [56]:
%%time
df['embedded_description'] = df['description'].apply(lambda x: list(get_prompt_embeddings(x)[0].cpu().flatten().numpy()))
df['embedded_description']

CPU times: user 2min 13s, sys: 299 ms, total: 2min 13s
Wall time: 2min 13s


0       [-0.002062, 0.0001404, 0.003252, 0.00869, -0.0...
1       [-0.05307, 0.03357, 0.03827, -0.004963, -0.058...
2       [-0.04034, 0.02864, 0.0352, 0.01851, -0.0503, ...
3       [-0.01447, 0.04117, 0.014435, 0.001357, -0.066...
4       [-0.01269, 0.02371, -0.02359, -0.00721, -0.024...
                              ...                        
9995    [-0.010056, 0.0513, 0.0211, 0.01463, -0.0674, ...
9996    [-0.01569, 0.01171, 0.02174, 0.01979, -0.03065...
9997    [-0.0463, 0.052, 0.04196, 0.01709, -0.06274, 0...
9998    [-0.05347, 0.03069, 0.04333, 0.012215, -0.0591...
9999    [-0.012146, 0.03958, -0.006634, 0.00769, -0.05...
Name: embedded_description, Length: 10000, dtype: object

In [57]:
len(df['embedded_description'][0])

768

 # sampling

In [67]:
X = np.array(list(df['embedded_description']))
y = np.array(list(df['priceRange']))
print(f'X shape: {X.shape} y shape: {y.shape}')

X shape: (10000, 768) y shape: (10000,)


In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=2025)

 # modeling

In [69]:
rf=RandomForestClassifier(
    random_state=2023,
    n_jobs=-1,
    class_weight='balanced'
)
#multilabel_et = OneVsOneClassifier(et)
multilabel_rf = OneVsRestClassifier(rf)

In [70]:
param_grid={
    'estimator__max_features':  ['sqrt', 'log2'],
    'estimator__max_depth' : range(5,25,5),
    'estimator__min_samples_leaf': [0.001,0.1,0.25,0.5],
    'estimator__min_samples_split': range(15,60,5),
    'estimator__criterion' :['gini', 'entropy'],
   'estimator__n_estimators': range(50,200,25),
}

In [71]:
%%time
CV_rfc = GridSearchCV(
    estimator=multilabel_rf, 
    param_grid=param_grid, 
    scoring='accuracy', 
    cv= 5
)
CV_rfc.fit(X_train, y_train)

CPU times: user 3d 22h 41min 34s, sys: 2h 45min 10s, total: 4d 1h 26min 45s
Wall time: 21h 28min 7s


GridSearchCV(cv=5,
             estimator=OneVsRestClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                                            n_jobs=-1,
                                                                            random_state=2023)),
             param_grid={'estimator__criterion': ['gini', 'entropy'],
                         'estimator__max_depth': range(5, 25, 5),
                         'estimator__max_features': ['sqrt', 'log2'],
                         'estimator__min_samples_leaf': [0.001, 0.1, 0.25, 0.5],
                         'estimator__min_samples_split': range(15, 60, 5),
                         'estimator__n_estimators': range(50, 200, 25)},
             scoring='accuracy')

In [ ]:
print("Best: %f using %s" % (CV_rfc.best_score_, CV_rfc.best_params_))

Best: 0.377857 using {'estimator__criterion': 'entropy', 'estimator__max_depth': 15, 'estimator__max_features': 'sqrt', 'estimator__min_samples_leaf': 0.001, 'estimator__min_samples_split': 50, 'estimator__n_estimators': 150}


In [ ]:
y_hat = CV_rfc.predict(X_test)
print("Accuracy for Random Forest on CV data: ",accuracy_score(y_test,y_hat))

Accuracy for Random Forest on CV data:  0.3586666666666667


In [ ]:
print("f1 score for Random Forest on CV data: ",f1_score(y_test,y_hat, average='micro'))

f1 score for Random Forest on CV data:  0.3586666666666667


In [ ]:
print(classification_report(y_test, y_hat))

              precision    recall  f1-score   support

           0       0.29      0.13      0.18       362
           1       0.47      0.49      0.48       550
           2       0.31      0.33      0.32       642
           3       0.30      0.32      0.31       698
           4       0.39      0.43      0.41       748

    accuracy                           0.36      3000
   macro avg       0.35      0.34      0.34      3000
weighted avg       0.35      0.36      0.35      3000



 # Save model

In [ ]:
timestamp = int(datetime.utcnow().timestamp())
pathfile = f'../data/{timestamp}_rf_op768.pkl'
print(f' Stored model: {pathfile}')
pickle.dump(CV_rfc, open(pathfile, "wb"))

 Stored model: ../data/1738214297_rf_op768.pkl


 # Validation
# local model

In [77]:
pkl_filename_local = '../data/1738214297_rf_op768.pkl'
with open(pkl_filename_local, 'rb') as file:
    uploaded_CV_rfc_local = pickle.load(file)

In [78]:
y_hat_dtc = uploaded_CV_rfc_local.predict(X_test)
print("Test f1_score: {0:.4f} %".format(100 * f1_score(y_test,y_hat_dtc, average='micro')))

Test f1_score: 35.8667 %


In [79]:
print("Test f1_score: {0:.4f} %".format(100 * f1_score(y_test,y_hat_dtc, average='macro')))

Test f1_score: 33.9577 %


In [80]:
print(classification_report(y_test, y_hat_dtc))

              precision    recall  f1-score   support

           0       0.29      0.13      0.18       362
           1       0.47      0.49      0.48       550
           2       0.31      0.33      0.32       642
           3       0.30      0.32      0.31       698
           4       0.39      0.43      0.41       748

    accuracy                           0.36      3000
   macro avg       0.35      0.34      0.34      3000
weighted avg       0.35      0.36      0.35      3000

